In [1]:
import pandas as pd

test_data = pd.read_csv('holdout_data.csv')
print(test_data.shape)
test_data.head()

(165, 1294)


,Info_PepID,Info_organism_id,Info_protein_id,Info_pos,Info_AA,Info_pubmed_id,Info_epitope_id,Info_host_id,Info_nPos,Info_nNeg,...,feat_esm1b_1271,feat_esm1b_1272,feat_esm1b_1273,feat_esm1b_1274,feat_esm1b_1275,feat_esm1b_1276,feat_esm1b_1277,feat_esm1b_1278,feat_esm1b_1279,Class
0,ABI20203.1:2,11676,ABI20203.1,263,K,23318151,33252,10000067,0,1,...,-0.140247,-0.248953,0.185531,0.007731,0.102163,-0.068917,-0.018686,-0.266192,0.080609,-1
1,ABI20203.1:2,11676,ABI20203.1,264,R,23318151,33252,10000067,0,1,...,-0.097484,-0.307103,0.462169,0.035631,0.138797,-0.006677,0.313747,-0.117993,-0.418884,-1
2,ABI20203.1:2,11676,ABI20203.1,265,W,23318151,33252,10000067,0,1,...,-0.004364,0.172317,0.146461,0.583348,0.155013,0.094847,-0.102562,-0.207383,-0.844568,-1
3,ABI20203.1:2,11676,ABI20203.1,266,I,23318151,33252,10000067,0,1,...,0.178255,-0.102600,0.305469,0.478632,0.283133,0.035063,0.006345,-0.213015,-0.269656,-1
4,ABI20203.1:2,11676,ABI20203.1,267,I,23318151,33252,10000067,0,1,...,0.274625,-0.207430,-0.009398,0.530498,0.324607,0.240791,0.149491,-0.069145,0.169097,-1


Need to make the protein sequences into Fasta format to allow for BepiPred 3.0 predictions 

In [2]:
unique_protein_ids = test_data['Info_protein_id'].unique()
print(unique_protein_ids)

['ABI20203.1' 'ABE68798.1' 'ACJ76655.1' 'AAN08336.1' 'AAS38336.1'
 'ADZ56462.1' 'CAY83124.1' 'ADZ56445.1' 'CAY83134.1' 'AAC03762.1'
 'P16082.1' 'ADU02647.1' 'P23426.1' 'AAC54646.1']


Need to make sure that the protein sequence is ordered for the most accurate prediction

In [3]:
test_data.sort_values(by=['Info_protein_id', 'Info_pos'], inplace=True)

# Create a dictionary to store the sequences for each Info_protein_id
protein_sequences = {}

# Iterate through unique Info_protein_id values
unique_protein_ids = test_data['Info_protein_id'].unique()
for protein_id in unique_protein_ids:
    # Filter the DataFrame for the current Info_protein_id
    protein_test_data = test_data[test_data['Info_protein_id'] == protein_id]
    
    # Check for missing Info_pos values
    expected_positions = set(range(1, protein_test_data['Info_pos'].max() + 1))
    actual_positions = set(protein_test_data['Info_pos'])
    
    # Remove rows with missing positions
    missing_positions = list(expected_positions - actual_positions)
    protein_test_data = protein_test_data[~protein_test_data['Info_pos'].isin(missing_positions)]
    
    # Create the sequence by joining Info_AA values in order
    sequence = ''.join(protein_test_data['Info_AA'])
    
    # Store the sequence in the dictionary
    protein_sequences[protein_id] = sequence

# Now, protein_sequences contains sequences for each unique Info_protein_id
for protein_id, sequence in protein_sequences.items():
    print(f'Info_protein_id: {protein_id}')
    print(f'Info_AA: {sequence}')
    print()

Info_protein_id: AAC03762.1
Info_AA: REITFIYKSSCTDSDHCQEYQCKKVNLNSSDSERQQVEETFNLIGCIERTHVFCHTG

Info_protein_id: AAC54646.1
Info_AA: PKLEPWKHP

Info_protein_id: AAN08336.1
Info_AA: VCERVL

Info_protein_id: AAS38336.1
Info_AA: PLWR

Info_protein_id: ABE68798.1
Info_AA: PRWP

Info_protein_id: ABI20203.1
Info_AA: KRWIILGLNKIVRMYGPKEPFRDYVDRFYK

Info_protein_id: ACJ76655.1
Info_AA: KVHG

Info_protein_id: ADU02647.1
Info_AA: NLDKIAEE

Info_protein_id: ADZ56445.1
Info_AA: I

Info_protein_id: ADZ56462.1
Info_AA: IVIQ

Info_protein_id: CAY83124.1
Info_AA: PEI

Info_protein_id: CAY83134.1
Info_AA: PIFS

Info_protein_id: P16082.1
Info_AA: SNSVRVEDVTNTAEYWGFK

Info_protein_id: P23426.1
Info_AA: TGKIPWILLPGR



In [4]:
for protein_id, sequence in protein_sequences.items():
    # Create a FASTA header line
    fasta_header = f'>{protein_id}\n'
    
    # Print the FASTA header and sequence
    print(fasta_header + sequence)

>AAC03762.1
REITFIYKSSCTDSDHCQEYQCKKVNLNSSDSERQQVEETFNLIGCIERTHVFCHTG
>AAC54646.1
PKLEPWKHP
>AAN08336.1
VCERVL
>AAS38336.1
PLWR
>ABE68798.1
PRWP
>ABI20203.1
KRWIILGLNKIVRMYGPKEPFRDYVDRFYK
>ACJ76655.1
KVHG
>ADU02647.1
NLDKIAEE
>ADZ56445.1
I
>ADZ56462.1
IVIQ
>CAY83124.1
PEI
>CAY83134.1
PIFS
>P16082.1
SNSVRVEDVTNTAEYWGFK
>P23426.1
TGKIPWILLPGR
